## 反向序列化

    将前台提交的数据传入Serializer的data参数中，进行实例化，可以得到一个反向序列化的实例对象。
    
    但是，要将该对象保存到数据库中，还必须调用.is_valid()方法进行数据验证。

## 数据验证
    
    Serializer的数据验证功能与django的form表单极其相似。
    
    Serializer中每个字段都有默认的验证器，当调用instance.is_valid()方法时，会根据每个字段的验证器对字段进行验证：
        倘若通过，返回True，所有通过验证的数据都将保存在instance.validated_data字典中。（instance.validated_data是一个property属性，其等于instance._validated_data）
        倘若失败，返回False，错误信息将保存到instance.errors字典中。
        
    以官方文档中的例子如下：

In [ ]:
serializer = CommentSerializer(data={'email': 'foobar', 'content': 'baz'})
serializer.is_valid()
# False
serializer.errors
# {'email': ['Enter a valid e-mail address.'], 'created': ['This field is required.']}

#### 验证单个字段

    倘若需要对某个字段进行额外的验证，则可以自定义验证方法，其方式与django的form表单类似。方法名称为：validate_<field_name>，该方法接收一个value参数，当原始的值通过字段默认验证之后，会被传入自定义验证方法中，验证结果应当如下：
    如果通过验证，则返回value值；
    如果未通过验证，则抛出异常serializers.ValidationError。
    
    官方文档示例如下：

In [ ]:
from rest_framework import serializers

class BlogPostSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100)
    content = serializers.CharField()

    def validate_title(self, value):
        """
        Check that the blog post is about Django.
        """
        if 'django' not in value.lower():
            raise serializers.ValidationError("Blog post is not about Django")
        return value

#### 验证多个字段

    倘若需要多个字段，比如需要确定用户输入的两次密码是否一致。此时可以自定义validate方法，该方法接收一个data参数，instance.validated_data会作为参数被传入该方法中。执行结果为：
    如果验证成功，则返回验证后的值，即传入的字典（可以对其中的值进行加工修改后返回）。
    如果验证失败，则抛出Serializers.ValidationError异常。
    
    官方文档示例：

In [ ]:
from rest_framework import serializers

class EventSerializer(serializers.Serializer):
    description = serializers.CharField(max_length=100)
    start = serializers.DateTimeField()
    finish = serializers.DateTimeField()

    def validate(self, data):
        """
        Check that start is before finish.
        """
        if data['start'] > data['finish']:
            raise serializers.ValidationError("finish must occur after start")
        return data

### 验证器
    
    单个除了上述方法对数据进行验证外，还可以定义一个单独的函数或者类，这种方式显得更加灵活，与Serializer解耦，有更强的复用性。
    
    与验证方法不同的地方在于，无论是字段中定义的validators还是Meta类中定义的validators，验证成功后均无需返回验证的值。而验证方法则需要返回验证成功后的值。

#### 字段级验证器 - 验证单个字段

    函数含一个value参数，用于接收对应字段的值。将该函数对象添加到字段的validators选项中。当执行验证时，会先执行字段的默认验证，再调用validators中的验证器。处理结果：
        如果验证成功，则无需返回验证的值
        如果验证失败，则引发serializers.ValidationError

    官方文档示例：

In [ ]:
def multiple_of_ten(value):
    if value % 10 != 0:
        raise serializers.ValidationError('Not a multiple of ten')

class GameRecord(serializers.Serializer):
    score = IntegerField(validators=[multiple_of_ten])
    ...

#### 对象级验证器 - 验证多个字段
    
    drf内置了很多可以在Meta类中配置的验证器（详情可参考官方文档）。这些验证器可以对多个字段进行依赖验证。
    
    官方文档示例如下：

In [ ]:
class EventSerializer(serializers.Serializer):
    name = serializers.CharField()
    room_number = serializers.IntegerField(choices=[101, 102, 103, 201])
    date = serializers.DateField()

    class Meta:
        # Each room only has one event per day.
        validators = [
            UniqueTogetherValidator(
                queryset=Event.objects.all(),
                fields=['room_number', 'date']
            )
        ]


#### 关联字段验证

    与序列化时的方式相同，将Serializer实例作为字段即可。倘若该字段允许为空，则required=False，字段可能关联多个对象时，设置many=True
    
    官方文档示例： 

In [2]:
class CommentSerializer(serializers.Serializer):
    user = UserSerializer(required=False)
    edits = EditItemSerializer(many=True)  # A nested list of 'edit' items.
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

## Serializer数据验证核心方法源码

In [ ]:
    # serializer.is_valid  -->  serializer.run_validation
    def run_validation(self, data=empty):
        """
        We override the default `run_validation`, because the validation
        performed by validators and the `.validate()` method should
        be coerced into an error dictionary with a 'non_fields_error' key.
        """
        (is_empty_value, data) = self.validate_empty_values(data)
        if is_empty_value:
            return data
        # 执行字段默认验证，再执行字段validators, 再执行validate_<field_name>
        value = self.to_internal_value(data)
        try:
            # 执行meta类中的validators
            self.run_validators(value)
            value = self.validate(value)
            assert value is not None, '.validate() should return the validated data'
        except (ValidationError, DjangoValidationError) as exc:
            raise ValidationError(detail=as_serializer_error(exc))
        
        # 此处的value即self.validated_data字典
        return value